Tensorflow exploration<br /><br />

Many variations were constructed.<br />
Best was Recall: 80.18<br />

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.10.1


In [11]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn

from imblearn.under_sampling import NearMiss 
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

In [3]:
def OverSample(df, target):
    over_sampler = SMOTE(random_state=42, sampling_strategy='minority')
    
    train_labels = df[target].copy()
    train_features = df.drop(columns=target)

    over_sampled_features, over_sampled_labels = over_sampler.fit_resample(train_features, train_labels)

    over_sampled_data = over_sampled_features
    over_sampled_data[target] = over_sampled_labels
    return over_sampled_data

In [4]:
df = pd.read_csv("..\diabetes_cleaned.csv")

#drop the old index, and heart specific as they are combined in heart_related
df.drop(['Unnamed: 0','heart_attack','angina_or_chd','chd_mi'], axis=1, inplace=True)

In [5]:
df = df.astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302953 entries, 0 to 302952
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   diabetes                  302953 non-null  float64
 1   bmi                       302953 non-null  float64
 2   smoker                    302953 non-null  float64
 3   stroke                    302953 non-null  float64
 4   asthma                    302953 non-null  float64
 5   physical_activity         302953 non-null  float64
 6   heavy_drinking            302953 non-null  float64
 7   no_doctor_due_to_cost     302953 non-null  float64
 8   any_healthcare_insurance  302953 non-null  float64
 9   general_health_status     302953 non-null  float64
 10  mental_health_status      302953 non-null  float64
 11  physical_health_status    302953 non-null  float64
 12  difficulty_walking        302953 non-null  float64
 13  gender                    302953 non-null  f

In [6]:
mpl.rcParams['figure.figsize'] = (6, 5)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [7]:
df.head()

,diabetes,bmi,smoker,stroke,asthma,physical_activity,heavy_drinking,no_doctor_due_to_cost,any_healthcare_insurance,general_health_status,...,difficulty_walking,gender,age,education,income,race,sleep_time,years_smoked,routine_checkup,heart_related
0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,...,0.0,0.0,7.0,2.0,3.0,1.0,6.0,0.0,0.0,1.0
1,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,...,0.0,0.0,4.0,4.0,6.0,1.0,5.0,0.0,1.0,1.0
2,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,4.0,...,0.0,0.0,3.0,3.0,3.0,1.0,9.0,0.0,1.0,1.0
3,0.0,2.0,3.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,...,0.0,0.0,7.0,4.0,5.0,2.0,7.0,0.0,1.0,1.0
4,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,...,0.0,0.0,7.0,2.0,5.0,1.0,8.0,0.0,1.0,1.0


In [8]:
neg, pre, dia = np.bincount(df['diabetes'])
total = neg + pre + dia

neg_percent = 100 * (neg/total)
pre_percent = 100 * (pre/total)
dia_percent = 100 * (dia/total)

print(f'Total: {total}\n   None: {neg} ({neg_percent:.2f}%)\n   Pre: {pre} ({pre_percent:.2f}%)\n   Diabetes: {dia} ({dia_percent:.2f}%)')

Total: 302953
   None: 254681 (84.07%)
   Pre: 6793 (2.24%)
   Diabetes: 41479 (13.69%)


In [9]:
#change to binary problem
binary_data = df
binary_data.loc[binary_data.diabetes.isin([2]), "diabetes"] = 1
binary_data.diabetes.value_counts()

diabetes
0.0    254681
1.0     48272
Name: count, dtype: int64

In [13]:
train_labels = df['diabetes'].copy()
train_features = df.drop(columns='diabetes')

over_sampler = RandomOverSampler(random_state=42)
X_over, y_over = over_sampler.fit_resample(train_features, train_labels)
sampled_data = X_over
sampled_data['diabetes'] = y_over
sampled_data.diabetes.value_counts()

diabetes
0.0    254681
1.0    254681
Name: count, dtype: int64

In [14]:
#We might need to reduce years smoked into categories
sampled_data.describe()


,bmi,smoker,stroke,asthma,physical_activity,heavy_drinking,no_doctor_due_to_cost,any_healthcare_insurance,general_health_status,mental_health_status,...,gender,age,education,income,race,sleep_time,years_smoked,routine_checkup,heart_related,diabetes
count,509362.000000,509362.000000,509362.000000,509362.000000,509362.000000,509362.000000,509362.000000,509362.000000,509362.000000,509362.000000,...,509362.000000,509362.000000,509362.000000,509362.000000,509362.000000,509362.000000,509362.000000,509362.000000,509362.0,509362.0
mean,3.179554,1.264869,0.062557,0.164172,0.718275,0.053993,0.080858,1.632093,2.794180,0.549906,...,0.502870,4.424229,3.050999,4.271371,1.323475,7.011283,1.120142,1.240414,1.0,0.5
std,0.815484,1.406730,0.242164,0.370432,0.449840,0.226004,0.272617,0.752717,1.073819,0.740387,...,0.499992,1.660777,0.932586,1.643439,0.820964,1.572981,9.347540,0.670002,0.0,0.5
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.0,0.0
25%,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,...,0.000000,3.000000,2.000000,3.000000,1.000000,6.000000,0.000000,1.000000,1.0,0.0
50%,3.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,3.000000,0.000000,...,1.000000,5.000000,3.000000,5.000000,1.000000,7.000000,0.000000,1.000000,1.0,0.5
75%,4.000000,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,3.000000,1.000000,...,1.000000,6.000000,4.000000,5.000000,1.000000,8.000000,0.000000,1.000000,1.0,1.0
max,4.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,5.000000,2.000000,...,1.000000,7.000000,4.000000,7.000000,6.000000,24.000000,99.000000,4.000000,1.0,1.0


In [15]:
# Use a utility from sklearn to split and shuffle your dataset.
train_df, test_df = train_test_split(sampled_data, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

In [16]:
train_df

,bmi,smoker,stroke,asthma,physical_activity,heavy_drinking,no_doctor_due_to_cost,any_healthcare_insurance,general_health_status,mental_health_status,...,gender,age,education,income,race,sleep_time,years_smoked,routine_checkup,heart_related,diabetes
503610,3.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,3.0,0.0,...,1.0,7.0,4.0,5.0,1.0,7.0,0.0,1.0,1.0,1.0
10577,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,...,0.0,7.0,4.0,5.0,1.0,6.0,0.0,1.0,1.0,0.0
482276,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,...,1.0,5.0,4.0,6.0,1.0,7.0,0.0,1.0,1.0,1.0
206894,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,...,0.0,5.0,3.0,5.0,1.0,6.0,0.0,1.0,1.0,0.0
122647,2.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,...,0.0,1.0,4.0,4.0,3.0,6.0,0.0,3.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82051,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,3.0,0.0,...,1.0,7.0,4.0,5.0,1.0,8.0,0.0,1.0,1.0,0.0
398032,4.0,3.0,0.0,0.0,0.0,1.0,0.0,2.0,3.0,0.0,...,0.0,6.0,4.0,5.0,1.0,8.0,0.0,2.0,1.0,1.0
52556,4.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,...,1.0,4.0,4.0,6.0,1.0,8.0,88.0,1.0,1.0,0.0
495299,4.0,3.0,1.0,0.0,0.0,0.0,0.0,2.0,4.0,2.0,...,1.0,6.0,3.0,3.0,1.0,4.0,0.0,1.0,1.0,1.0


In [17]:
# Form np arrays of labels and features.
train_labels = np.array(train_df['diabetes'])

neg_train_labels = train_labels == 0
pre_train_labels = train_labels == 1
dia_train_labels = train_labels == 2

val_labels = np.array(val_df['diabetes'])
test_labels = np.array(test_df['diabetes'])

In [18]:
train_features = np.array(train_df.drop(columns='diabetes'))
val_features = np.array(val_df.drop(columns='diabetes'))
test_features = np.array(test_df.drop(columns='diabetes'))

In [19]:
print(f'Average class probability in training set:   {train_labels.mean():.4f}')
print(f'Average class probability in validation set: {val_labels.mean():.4f}')
print(f'Average class probability in test set:       {test_labels.mean():.4f}')

Average class probability in training set:   0.5001
Average class probability in validation set: 0.4977
Average class probability in test set:       0.5016


In [20]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

Training labels shape: (325991,)
Validation labels shape: (81498,)
Test labels shape: (101873,)
Training features shape: (325991, 21)
Validation features shape: (81498, 21)
Test features shape: (101873, 21)


In [18]:
#sns.jointplot(data=df, x="age", y="sleep_time", hue="diabetes")

In [19]:
#sns.jointplot(data=df, x="age", y="sleep_time", hue="diabetes", kind="kde")

In [20]:
#sns.jointplot(data=df, x="age", y="heart_related", hue="diabetes")

In [21]:
# neg_df = pd.DataFrame(train_features[neg_train_labels ], columns=train_df.columns)
# prediabetes_df = pd.DataFrame(train_features[ pre_train_labels], columns=train_df.columns)
# diabetes_df = pd.DataFrame(train_features[dia_train_labels], columns=train_df.columns)

# sns.jointplot(x=diabetes_df['age'], y=diabetes_df['sleep_time'],
#               kind='hex', xlim=(-3,3), ylim=(-5,5))
# plt.suptitle("Positive distribution")

# sns.jointplot(x=neg_df['age'], y=neg_df['sleep_time'],
#               kind='hex', xlim=(-3,3), ylim=(-5,5))
# _ = plt.suptitle("Negative distribution")

In [21]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)


In [41]:
layer_sizes = [2000]
for layer_size in layer_sizes:
    feature_size = train_features.shape[-1]
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(layer_size, activation='relu',input_shape=(train_features.shape[-1],)),
        keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(500, activation='relu'),
        keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(100, activation='relu'),
        keras.layers.Dropout(0.2),


        # Classification  of 3 
        # #tf.keras.layers.Dense(3, activation='softmax')  
        
        #Binary
        tf.keras.layers.Dense(1,activation="sigmoid")  
    ])
    model.summary()

    model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                metrics=['Recall']
                )

    model.fit(train_features, train_labels, callbacks=[callback], epochs=20)

    #test_loss, test_acc = model.evaluate(test_features,  test_labels, verbose=2)
    predictions = model.predict(test_features)
    
    predicted = np.array([1 if x >= 0.5 else 0 for x in predictions])

    recall_score(test_labels, predicted)

    

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 2000)              44000     
                                                                 
 dropout_24 (Dropout)        (None, 2000)              0         
                                                                 
 dense_39 (Dense)            (None, 1000)              2001000   
                                                                 
 dropout_25 (Dropout)        (None, 1000)              0         
                                                                 
 dense_40 (Dense)            (None, 500)               500500    
                                                                 
 dropout_26 (Dropout)        (None, 500)               0         
                                                                 
 dense_41 (Dense)            (None, 100)             

Epoch 1/20
10188/10188 [==============================] - 59s 6ms/step - loss: 0.5456 - recall: 0.7962
Epoch 2/20
10188/10188 [==============================] - 55s 5ms/step - loss: 0.5424 - recall: 0.7994
Epoch 3/20
10188/10188 [==============================] - 56s 5ms/step - loss: 0.5414 - recall: 0.7990
Epoch 4/20
10188/10188 [==============================] - 55s 5ms/step - loss: 0.5408 - recall: 0.7968
Epoch 5/20
10188/10188 [==============================] - 56s 5ms/step - loss: 0.5402 - recall: 0.7946
Epoch 6/20
10188/10188 [==============================] - 56s 6ms/step - loss: 0.5391 - recall: 0.8004
Epoch 7/20
10188/10188 [==============================] - 55s 5ms/step - loss: 0.5389 - recall: 0.7968
Epoch 8/20
10188/10188 [==============================] - 57s 6ms/step - loss: 0.5386 - recall: 0.7949
Epoch 9/20
10188/10188 [==============================] - 55s 5ms/step - loss: 0.5385 - recall: 0.7932
Epoch 10/20
10188/10188 [==============================] - 55s 5ms/step -

0.8196692435659066
Layer (type)                Output Shape              Param #   
=================================================================
 dense_28 (Dense)            (None, 2000)              44000     
                                                                 
 dropout_16 (Dropout)        (None, 2000)              0         
                                                                 
 dense_29 (Dense)            (None, 1000)              2001000   
                                                                 
 dropout_17 (Dropout)        (None, 1000)              0         
                                                                 
 dense_30 (Dense)            (None, 500)               500500    
                                                                 
 dropout_18 (Dropout)        (None, 500)               0         
                                                                 
 dense_31 (Dense)            (None, 100)               50100     
                                                                 
 dropout_19 (Dropout)        (None, 100)               0         
                                                                 
 dense_32 (Dense)            (None, 1)                 101   

In [43]:
    recall_score(test_labels, predicted)

0.8018397103434778